In [ ]:
##### This code is used for obtaining rate model data for different values oof neuronal loss D_const #####
if 'RunningFromFigureScript' in locals(): # Checking where from the script was runned. If from
    # figure, the directory path is then different
    %run ./DataGenerationScripts/load_default_parameters.ipynb # loading model parameters
    %run ./DataGenerationScripts/dIBDRdt_functions.ipynb # loading time derivative functions
else:
    %reset 
    %run ../DataGenerationScripts/load_default_parameters.ipynb # loading model parameters
    %run ../DataGenerationScripts/dIBDRdt_functions.ipynb # loading time derivative functions 
    
import numpy as np
from tqdm import tqdm

In [ ]:
D0_vect=[0.45, 0.5, 0.6, 0.7, 0.8, 0.9, 1.]

In [ ]:
for ww in range(0,len(D0_vect)):
    params=load_default_parameters('rate') # here we define simulation-specific parameters of simulation duration; external input parameters; number of simulation; output location for data  
    params['t_end']= 365*40 # 40 years          
    params['IBDR_E_duration']= [0,0,0,0]
    params['IBDR_E_amplitude']= [0,0,0,0]
    params['Complex_input']= 'no'
    params['amount_simulations']=  len(D0_vect)
    params['number_simulation']= ww+1
    params['IC']=[0,0,D0_vect[ww],0] # here we set the D0 as initial condition for D(t)
    if 'RunningFromFigureScript' in locals(): # Checking where from the script was runned and creating respective figure path
        params['path']= './Data/Data_F7_Rate_NeurLossm_Dvar/Sim_'
    else:
        params['path']= '../Data/Data_F7_Rate_NeurLossm_Dvar/Sim_'

    locals().update(params)

    t_vec=np.linspace(0,t_end,int(t_end/dt+1))
    I_vec=np.zeros(np.shape(t_vec))
    B_vec=np.zeros(np.shape(t_vec))
    D_vec=np.zeros(np.shape(t_vec))
    R_vec=np.zeros(np.shape(t_vec))

    t_lastseiz = []
    I_vec[0]=IC[0] #  setting initial conditions
    B_vec[0]=IC[1]
    D_vec[0]=IC[2]
    R_vec[0]=IC[3]

    for ii in tqdm(range(1, len(t_vec))): # solving with forward Euler method  

        [dIdt, dBdt, dDdt, dRdt] = \
                    dIBDRdt_Rate(t_vec[ii], I_vec[ii-1], B_vec[ii-1], D_vec[ii-1], R_vec[ii-1], params)

        I_vec[ii] = I_vec[ii-1] + dIdt * dt
        B_vec[ii] = B_vec[ii-1] + dBdt * dt
        D_vec[ii] = D_vec[ii-1] + dDdt * dt
        R_vec[ii] = R_vec[ii-1] + dRdt * dt


    results_dict = {'t_vec': t_vec,
                    'I_vec': I_vec,
                    'B_vec': B_vec,
                    'D_vec': D_vec,
                    'R_vec': R_vec}

    filename = path + str(number_simulation)
    np.save(filename, [params, results_dict])
    del ii, filename